# cx_Oracle 모듈
- 파이썬에서 오라클 DBMS와 연동는 다양한 함수를 제공하는 모듈
- Python DB API 2.0 표준을 따른다. 

## 참고
- [cx_Oracle 모듈 홈페이지](https://oracle.github.io/python-cx_Oracle/)
- [Python DB API](https://docs.python.org/ko/3/library/sqlite3.html)

## cx_Oracle 모듈 설치
- [설치 튜토리얼](https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html)
- 패키지 설치
```
conda install cx_oracle
pip install cx_Oracle --upgrade
```

## 코딩 절차
1. cx_Oracle 모듈 import

2. cx_Oracle모듈의 connect() 함수를 이용해 디비연결

3. Connection 객체의 cursor() 메소드를 이용해 Cursor객체 조회.
    - Cursor객체: sql문을 전송하고 select결과 조회 기능을 제공하는 객체

4. Cursor 의 execute() 메소드를 이용해 SQL문 전송

5. SELECT 의 경우 결과 조회.
    - 조회결과 한행씩 반환받아 사용
    - 조회결과 한번에 반환받아 사용

6. INSERT/DELETE/UPDATE의 경우  SQL 문 실행 후 Connection의 commit() 메소드를 이용해 Transaction 처리

7. Cursor, Connection 연결 닫기

### DB 연결
- connect() 함수를 이용하며 연결 후 Connection 객체를 받는다.
- 연결시 필요한 값
    - **host**
    - **port번호**
    - **SID (DB이름)** -> 오라클에서는 SID, 다른 곳에서는 DB이름
    - **user name**
    - **password**

### Connection 주요 메소드
- commit(): 커밋
- rollback(): 롤백
- close(): 연결 닫기

In [7]:
# 방법 1

# connect 함수를 통해 연결. 연결(접속)에 성공하면 Connection 객체를 반환
# Connection 객체: 연결정보를 가지고 있는 객체
import cx_Oracle

username = 'c##scott_join'
password = 'tiger'
url = 'localhost:51521/XE' # host:port/sid 의 규격
conn = cx_Oracle.connect(username, password, url) # 연결
print(type(conn))
conn.close()

<class 'cx_Oracle.Connection'>


In [8]:
# 방법 2

# username/pwd@url
conn2 = cx_Oracle.connect('c##scott_join/tiger@localhost:51521/XE')
print(type(conn2))
conn2.close()

<class 'cx_Oracle.Connection'>


#### DSN (Data Source Name) 을 이용한 연결
- DSN: DBMS를 연결하기 위한 정보를 모아 놓은 것. (서로 다른 DBMS의 연결정보를 통일하기 위해서 방식)
- makedsn(host, port번호, SID) 함수 이용해 생성한다.

In [11]:
# 방법 3

username = 'c##scott_join'
password = 'tiger'
host = 'localhost'
port = 51521
sid = 'XE'

dsn = cx_Oracle.makedsn(host, port, sid)
print(dsn)
conn3 = cx_Oracle.connect(username, password, dsn)
print(type(conn3))
conn3.close()

(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=51521))(CONNECT_DATA=(SID=XE)))
<class 'cx_Oracle.Connection'>


In [13]:
# with 블록을 사용했기 때문에 자동으로 close처리를 수행한다.
with cx_Oracle.connect(username, password, dsn) as conn4:
    # DB 관련 작업     ex) conn4.xxxx
    pass

## Cursor 객체
- SQL문을 전송하고 select결과를 조회하는 메소드들을 제공한다.
- Connection객체의 cursor() 메소드로 받아온다.

### Cursor의 주요 메소드
#### SQL 실행 메소드
- execute(sql) : 하나의 sql 문 실행. 
- executemany(sql): insert, update, delete 배치 처리
    - insert, update, delete는 다 처리후 conn.commit()으로 커밋 처리해야한다.
    

#### select 결과 조회 메소드
- execute()로 실행한 SQL이 select인 경우 다음 메소드로 조회한다.
- fetchall()  : 조회된 모든 행을 한번에 가져올 때 사용한다. 결과를 tuple들을 묶은 리스트로 반환
- fetchone()  : 호출시 마다 한행씩 반환한다. PK로 조회한 경우 많이 사용한다.
- fetchmany(n): n행만큼 조회한다. n기본값-100. 특정개수반큼 반복문을 이용해 가져올때 사용.

In [14]:
import cx_Oracle
from pprint import pprint  # 자료구조를 보기 좋게 출력해주는 print 함수

In [29]:
# SQL문: 문자열로 작성, 대소문자 구분 X (sql의 keyword들은 보통 대문자, 컬럼명/테이블명은 소문자)
#     : ';'은 붙이지 않는다.
select_sql = 'SELECT emp_id, emp_name, job_id FROM emp'
insert_sql = "INSERT INTO emp VALUES (1510, '홍길동', 'FI_ACCOUNT', 101, to_date('2021/02/15', 'yyyy/mm/dd'), 30000, 0.2, 100)"
conn = None  # Connection을 대입할 변수
cursor = None  # Cursor 객체를 대입할 변수

# INSERT 후에 SELECT
try:
    # 1. 연결
    conn = cx_Oracle.connect('c##scott_join/tiger@localhost:51521/XE')
    print('연결 완료')
    # 2. cursor 생성
    cursor = conn.cursor()
    # 3. sql 실행
    cursor.execute(insert_sql)
    cursor.execute(select_sql)
    # 4. select 결과 조회
    result = cursor.fetchall()  # 조회 결과를 한 번에 반환. 한 행 - tuple로 반환 / 여러 행 - list로 tuple들을 묶어서 반환
    print(type(result), type(result[0]))
    print(len(result))
    pprint(result)
    # 5. insert, update, delete를 실행한 경우 commit 처리
    conn.commit()
    
except Exception as e:
    print(e)
    
finally:
    # 6. 연결 닫기 - cursor / connection
    if cursor != None:
        cursor.close()
    if conn != None:
        conn.close()
    

연결 완료
<class 'list'> <class 'tuple'>
113
[(100, 'Steven', 'AD_PRES'),
 (101, 'Neena', 'AD_VP'),
 (102, 'Lex', 'AD_VP'),
 (103, 'Alexander', 'IT_PROG'),
 (104, 'Bruce', 'IT_PROG'),
 (105, 'David', 'IT_PROG'),
 (106, 'Valli', 'IT_PROG'),
 (107, 'Diana', 'IT_PROG'),
 (108, 'Nancy', 'FI_MGR'),
 (109, 'Daniel', 'FI_ACCOUNT'),
 (110, 'John', 'FI_ACCOUNT'),
 (111, 'Ismael', 'FI_ACCOUNT'),
 (112, 'Jose Manuel', 'FI_ACCOUNT'),
 (113, 'Luis', 'FI_ACCOUNT'),
 (114, 'Den', 'PU_MAN'),
 (115, 'Alexander', 'PU_MAN'),
 (116, 'Shelli', 'PU_CLERK'),
 (117, 'Sigal', 'PU_CLERK'),
 (118, 'Guy', 'PU_CLERK'),
 (119, 'Karen', 'PU_CLERK'),
 (120, 'Matthew', 'ST_MAN'),
 (121, 'Adam', 'ST_MAN'),
 (122, 'Payam', 'ST_MAN'),
 (123, 'Shanta', 'ST_MAN'),
 (124, 'Kevin', 'ST_MAN'),
 (125, 'Julia', 'ST_CLERK'),
 (126, 'Irene', 'ST_CLERK'),
 (127, 'James', 'ST_CLERK'),
 (128, 'Steven', 'ST_CLERK'),
 (129, 'Laura', 'ST_CLERK'),
 (130, 'Mozhe', None),
 (131, 'James', None),
 (132, 'TJ', 'ST_CLERK'),
 (133, 'Jason', 'ST_CL

## placeholder(자리표시자)를 이용해 sql 실행
- SQL에 값이 들어갈 자리에 값을 대신할 문자 **`:순번` 또는 `:이름`** 를 넣고 SQL 실행시 값을 전달
> placehold 는 DBMS마다 다르다. 
- execute(SQL, placeholder에 들어갈 값)
    - 딕셔너리: placeholder의 이름-값 형식
    - 리스트나 튜플: placeholder 순서

In [30]:
import cx_Oracle
from pprint import pprint

In [35]:
select_sql = 'SELECT * FROM emp WHERE salary BETWEEN :1 AND :2'  # :1, :2 - placeholder(여기 들어갈 값은 나중에 지정)
insert_sql = "INSERT INTO emp VALUES\
(:id, :name, :job_id, :mgr_id, to_date(:hire_date, 'yyyymmdd'), :salary, :comm_pct, :dept_id)"

conn = None
cursor = None

try:
    conn = cx_Oracle.connect('c##scott_join/tiger@localhost:51521/XE')
    cursor = conn.cursor()
    
    # sql문 실행
    # varchar2 / char / nvarchar / nchar - str
    # number(정수 / 실수) - int / double
    # date - datetime 객체
    # null - none
    param_dict = {
        'id': 2000,
        'name': '이순신',
        'job_id': 'FI_ACCOUNT',
        'mgr_id': 101,
        'hire_date': '20210223',
        'salary': 19990,
        'comm_pct': None,
        'dept_id': 110
    }
    cursor.execute(insert_sql, param_dict)
    
    cursor.execute(select_sql, [15000, 20000])  # [10000, 20000]의 첫번째 값은 :1자리, 두번째 값은 :2자리
    result1 = cursor.fetchall()
    print('15000 - 20000 사이 salary 조회 ********************')
    pprint(result1)
    
    cursor.execute(select_sql, [2000, 3000])
    result2 = cursor.fetchall()
    print('2000 - 3000 사이 salary 조회 ********************')
    pprint(result2)
    
    conn.commit()
    
except Exception as e:
    print(e)
    
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

15000 - 20000 사이 salary 조회 ********************
[(101,
  'Neena',
  'AD_VP',
  100,
  datetime.datetime(2005, 9, 21, 0, 0),
  17000.0,
  None,
  90),
 (102,
  'Lex',
  'AD_VP',
  100,
  datetime.datetime(2001, 1, 13, 0, 0),
  17000.0,
  None,
  90),
 (107,
  'Diana',
  'IT_PROG',
  103,
  datetime.datetime(2007, 2, 7, 0, 0),
  18900.0,
  None,
  60),
 (108,
  'Nancy',
  'FI_MGR',
  100,
  datetime.datetime(2002, 8, 17, 0, 0),
  15008.0,
  0.2,
  100),
 (2000,
  '이순신',
  'FI_ACCOUNT',
  101,
  datetime.datetime(2021, 2, 23, 0, 0),
  19990.0,
  None,
  110)]
2000 - 3000 사이 salary 조회 ********************
[]


## SELECT  결과 조회

### fetchone()
- 조회결과 한행씩 반환한다.
- PK로 조회시 사용한다.
- 반환 type : tuple

### fetchall()
- 조회한 전체 결과셋을 반환한다.
- 한 row를 Tuple로 묶고 그 Tuple들을 List로 묶어서 반환한다.

### fetchmany(n)
- n 행씩 조회할 때 사용
- n 기본값 100

In [38]:
# emp_id(PK)로 직원 정보를 조회하는 함수
def select_emp_by_id(emp_id):
    sql = "SELECT * FROM emp WHERE emp_id = :1"

    with cx_Oracle.connect('c##scott_join/tiger@localhost:51521/XE') as conn:
        with conn.cursor() as cursor:
            # sql 실행
            cursor.execute(sql, [emp_id])
            result = cursor.fetchone()  # 한 행 조회(tuple). 조회결과가 없으면 None을 반환
            return result

In [41]:
r = select_emp_by_id(100)
if r != None:
    print(r)
else:
    print('조회결과가 없습니다.')

(100, 'Steven', 'AD_PRES', None, datetime.datetime(2003, 6, 17, 0, 0), 24000.0, None, 90)


In [42]:
r = select_emp_by_id(3000)
if r != None:
    print(r)
else:
    print('조회결과가 없습니다.')

조회결과가 없습니다.


In [50]:
sql = 'SELECT * FROM emp ORDER BY emp_id'
with cx_Oracle.connect('c##scott_join/tiger@localhost:51521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchmany(10)  # 10개만 조회
        print(len(result))
        print(result)
        
        result = cursor.fetchmany(10)  # 다음 10개
        print(len(result))
        print(result)

10
[(100, 'Steven', 'AD_PRES', None, datetime.datetime(2003, 6, 17, 0, 0), 24000.0, None, 90), (101, 'Neena', 'AD_VP', 100, datetime.datetime(2005, 9, 21, 0, 0), 17000.0, None, 90), (102, 'Lex', 'AD_VP', 100, datetime.datetime(2001, 1, 13, 0, 0), 17000.0, None, 90), (103, 'Alexander', 'IT_PROG', 102, datetime.datetime(2006, 1, 3, 0, 0), 27000.0, None, 60), (104, 'Bruce', 'IT_PROG', 103, datetime.datetime(2007, 5, 21, 0, 0), 27000.0, None, 60), (105, 'David', 'IT_PROG', 103, datetime.datetime(2005, 6, 25, 0, 0), 21600.0, None, 60), (106, 'Valli', 'IT_PROG', 103, datetime.datetime(2006, 2, 5, 0, 0), 21600.0, None, 60), (107, 'Diana', 'IT_PROG', 103, datetime.datetime(2007, 2, 7, 0, 0), 18900.0, None, 60), (108, 'Nancy', 'FI_MGR', 100, datetime.datetime(2002, 8, 17, 0, 0), 15008.0, 0.2, 100), (109, 'Daniel', 'FI_ACCOUNT', 100, datetime.datetime(2002, 8, 16, 0, 0), 12000.0, 0.2, 100)]
10
[(110, 'John', 'FI_ACCOUNT', 100, datetime.datetime(2005, 9, 28, 0, 0), 11200.0, 0.2, 100), (111, 'Isma

### cursor를 for in 문을 이용해 select결과 조회
- select후 cursor는 Iterable 한 객체.
- cursor는 반복시마다 조회한 한행을 반환.

In [59]:
sql = 'SELECT * FROM emp WHERE comm_pct IS NOT NULL ORDER BY emp_id'
with cx_Oracle.connect('c##scott_join/tiger@localhost:51521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(sql)
        
        # 한 행씩 처리하는 경우
        for idx, row in enumerate(cursor, start = 1):  # 한번 반복할 때마다 cursor.fetchone()을 수행
            print(f'{idx}. {row}')
            
        """
        r = cursor.fetchone()
        while r!= None:
            print(r)
            print('*' * 30)
            r = cursor.fetchone()
        """

1. (108, 'Nancy', 'FI_MGR', 100, datetime.datetime(2002, 8, 17, 0, 0), 15008.0, 0.2, 100)
2. (109, 'Daniel', 'FI_ACCOUNT', 100, datetime.datetime(2002, 8, 16, 0, 0), 12000.0, 0.2, 100)
3. (110, 'John', 'FI_ACCOUNT', 100, datetime.datetime(2005, 9, 28, 0, 0), 11200.0, 0.2, 100)
4. (111, 'Ismael', 'FI_ACCOUNT', 100, datetime.datetime(2005, 9, 30, 0, 0), 10700.0, 0.2, 100)
5. (112, 'Jose Manuel', 'FI_ACCOUNT', 100, datetime.datetime(2006, 3, 7, 0, 0), 10800.0, 0.2, 100)
6. (113, 'Luis', 'FI_ACCOUNT', 100, datetime.datetime(2007, 12, 7, 0, 0), 9900.0, 0.2, 100)
7. (149, 'Eleni', 'SA_MAN', 100, datetime.datetime(2007, 10, 15, 0, 0), 10500.0, 0.2, 80)
8. (151, 'David', 'SA_REP', 145, datetime.datetime(2005, 3, 24, 0, 0), 9500.0, 0.25, 80)
9. (152, 'Peter', 'SA_REP', 145, datetime.datetime(2005, 8, 20, 0, 0), 9000.0, 0.25, 80)
10. (153, 'Christopher', 'SA_REP', 145, datetime.datetime(2006, 3, 30, 0, 0), 8000.0, 0.2, 80)
11. (154, 'Nanette', 'SA_REP', 145, datetime.datetime(2006, 12, 9, 0, 0),

In [61]:
sql = "SELECT job_id, sum(salary), round(avg(salary), 2) FROM emp GROUP BY job_id"

with cx_Oracle.connect('c##scott_join/tiger@localhost:51521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchall()
        pprint(result)

[('AD_VP', 34000, 17000),
 ('FI_ACCOUNT', 156090, 14190),
 ('PU_CLERK', 16200, 4050),
 ('SH_CLERK', 83850, 4932.35),
 ('HR_REP', 9750, 9750),
 ('PU_MAN', 20100, 10050),
 ('AC_MGR', 12008, 12008),
 ('ST_CLERK', 71850, 4226.47),
 ('AD_ASST', 8250, 8250),
 ('IT_PROG', 116100, 23220),
 (None, 33300, 4757.14),
 ('SA_MAN', 61000, 12200),
 ('AC_ACCOUNT', 8300, 8300),
 ('FI_MGR', 15008, 15008),
 ('ST_MAN', 42550, 8510),
 ('AD_PRES', 24000, 24000),
 ('MK_MAN', 13000, 13000),
 ('SA_REP', 262950, 8765),
 ('MK_REP', 9000, 9000),
 ('PR_REP', 10000, 10000)]


In [62]:
sql = "SELECT e.emp_name, d.dept_name FROM emp e LEFT JOIN dept d ON e.dept_id = d.dept_id "

with cx_Oracle.connect('c##scott_join/tiger@localhost:51521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchall()
        pprint(result)

[('Michael', 'Marketing'),
 ('Pat', 'Marketing'),
 ('홍길동', 'Marketing'),
 ('Den', 'Purchasing'),
 ('Alexander', 'Purchasing'),
 ('Shelli', 'Purchasing'),
 ('Sigal', 'Purchasing'),
 ('Guy', 'Purchasing'),
 ('Karen', 'Purchasing'),
 ('Susan', 'Human Resources'),
 ('Matthew', 'Shipping'),
 ('Adam', 'Shipping'),
 ('Payam', 'Shipping'),
 ('Shanta', 'Shipping'),
 ('Kevin', 'Shipping'),
 ('Julia', 'Shipping'),
 ('Irene', 'Shipping'),
 ('James', 'Shipping'),
 ('Steven', 'Shipping'),
 ('Laura', 'Shipping'),
 ('Mozhe', 'Shipping'),
 ('James', 'Shipping'),
 ('TJ', 'Shipping'),
 ('Jason', 'Shipping'),
 ('Michael', 'Shipping'),
 ('Ki', 'Shipping'),
 ('Hazel', 'Shipping'),
 ('Renske', 'Shipping'),
 ('Stephen', 'Shipping'),
 ('John', 'Shipping'),
 ('Joshua', 'Shipping'),
 ('Trenna', 'Shipping'),
 ('Curtis', 'Shipping'),
 ('Randall', 'Shipping'),
 ('Peter', 'Shipping'),
 ('Winston', 'Shipping'),
 ('Jean', 'Shipping'),
 ('Martha', 'Shipping'),
 ('Julia', 'Shipping'),
 ('Anthony', 'Shipping'),
 ('Kelly'

## INSERT / UPDATE/ DELETE
- SQL 실행 후 Connection객체의 commit()을 호출 해야 최종적으로 적용된다.
- commit()
    - 최종 적용
- rollback()
    - 변경전 상태로 되돌리기

### 사용할 테이블
```sql
create table test(
    id varchar2(10),
    name varchar2(100),
    age number(3));
```

In [67]:
create_sql = 'create table test(id varchar2(10), name varchar2(100), age number(3))'
insert_sql = 'INSERT INTO test VALUES (:1, :2, :3)'

with cx_Oracle.connect('c##scott_join/tiger@localhost:51521/XE') as conn:
    with conn.cursor() as cursor:
        try:
            cursor.execute(create_sql)  # 테이블 생성
        except Exception as e:
            print(e)
        # insert
        cursor.execute(insert_sql, [1, '이름1', 20])
        cursor.execute(insert_sql, [2, '이름2', 30])
        cursor.execute(insert_sql, [3, '이름3', 40])
        conn.commit()  # commit

ORA-00955: name is already used by an existing object


In [69]:
delete_sql = "DELETE FROM test WHERE name = :1"

with cx_Oracle.connect('c##scott_join/tiger@localhost:51521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(delete_sql, ['이름1'])
        conn.commit()

In [71]:
update_sql = 'UPDATE test SET name = :1'

with cx_Oracle.connect('c##scott_join/tiger@localhost:51521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(update_sql, ['새이름'])
        conn.commit()

In [84]:
select_sql = 'SELECT * FROM test'

with cx_Oracle.connect('c##scott_join/tiger@localhost:51521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(select_sql)
        pprint(cursor.fetchall())

[('2', '새이름', 30),
 ('3', '새이름', 40),
 ('a-1', '이름1', 20),
 ('2', '새이름', 30),
 ('3', '새이름', 40),
 ('a-2', '이름2', 30),
 ('a-3', '이름3', 40),
 ('a-4', '이름4', 50),
 ('a-1', '이름1', 20),
 ('a-2', '이름2', 30),
 ('a-3', '이름3', 40),
 ('a-4', '이름4', 50)]


## batch(일괄작업) 처리
- 한번에 메소드 호출로 다수 행을 처리한다.
- executemany(sql, placeholder에 전달할값)
    - placeholder에 전달할 값을 list로 묶어서 전달하면 sql문이 그 개수만큼 한번에 처리된다.

In [83]:
insert_sql = "INSERT INTO test VALUES (:1 ,:2, :3)"
param = [
    ['a-1', '이름1', 20],
    ['a-2', '이름2', 30],
    ['a-3', '이름3', 40],
    ['a-4', '이름4', 50]
]

with cx_Oracle.connect('c##scott_join/tiger@localhost:51521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.executemany(insert_sql, param)
        conn.commit()

## TODO : 매개변수로 dept_id를 받아서 EMP 테이블에서 그 부서의 직원들의 salary를 두배 update하는 함수.

In [85]:
def update_salary_by_dept_id(dept_id):
    update_sql = "UPDATE emp SET salary = 2 * salary WHERE dept_id = :1"
    
    with cx_Oracle.connect('c##scott_join/tiger@localhost:51521/XE') as conn:
        with conn.cursor() as cursor:
            cursor.execute(update_sql, [dept_id])
            conn.commit()

## TODO : 매개변수로 job_id를 받아서 그 업무를 하는 직원들을 EMP 테이블에서 삭제하는 함수.

In [10]:
def delete_people_by_job_id(job_id):
    if job_id != None:
        delete_sql = "DELETE FROM emp WHERE job_id = :1"
    else:
        delete_sql = "DELETE FROM emp WHERE job_id IS NULL"  # is null을 사용해야하는 경우에는 이런 식으로
    
    with cx_Oracle.connect('c##scott_join/tiger@localhost:51521/XE') as conn:
        with conn.cursor() as cursor:
            if job_id != None:
                cursor.execute(delete_sql, [job_id])
            else:
                cursor.execute(delete_sql)  # 이 부분도 마찬가지로 if문으로 분리해줘야 한다.
            conn.commit()

# TODO : 매개변수로 직원 이름을 입력받아 EMP 테이블에서 그  직원들의 전체 정보를 조회결과를 출력하는 함수.

In [14]:
import cx_Oracle

def print_all_by_emp_name(emp_name):
    sql = "SELECT * FROM emp WHERE emp_name = :1"
    sql = "SELECT * FROM emp WHERE emp_name LIKE '%' || :1 || '%'"
    
    with cx_Oracle.connect('c##scott_join/tiger@localhost:51521/XE') as conn:
        with conn.cursor() as cursor:
            cursor.execute(sql, [emp_name])
            result = cursor.fetchall()
            return result

In [15]:
r = print_all_by_emp_name('Jo')
print(r)

[(110, 'John', 'FI_ACCOUNT', 100, datetime.datetime(2005, 9, 28, 0, 0), 11200.0, 0.2, 100), (112, 'Jose Manuel', 'FI_ACCOUNT', 100, datetime.datetime(2006, 3, 7, 0, 0), 10800.0, 0.2, 100), (139, 'John', 'ST_CLERK', 123, datetime.datetime(2008, 2, 6, 0, 0), 4050.0, None, 50), (140, 'Joshua', None, 123, datetime.datetime(2008, 2, 6, 0, 0), 3750.0, None, 50), (145, 'John', 'SA_MAN', 100, datetime.datetime(2004, 10, 1, 0, 0), 14000.0, None, 80), (176, 'Jonathon', 'SA_REP', 149, datetime.datetime(2006, 3, 24, 0, 0), 8600.0, 0.2, 80)]


# TODO: 사용자로 부터 직원정보를 입력받아 EMP 테이블에 입력한다.
- emp_id, emp_name, job_id, mgr_id, hire_date, salary, comm_pct, dept_id 를 하나씩 입력받는다.
- job_id, mgr_id, comm_pct, dept_id 을 입력하지 않은 경우 NULL 을 넣는다. (placehold 사용시 None을 대입)
- 입력된 값을 EMP 테이블에 insert 한다.
- 주의 job_id, mgr_id, dept_id는 FK 컬럼이므로 부모테이블의 PK값을 넣어야 한다.

In [1]:
def insert_info_by_input(emp_id, emp_name, hire_date, salary, job_id = None, mgr_id = None, comm_pct = None, dept_id = None):
    insert_sql = "INSERT INTO emp VALUES (:emp_id, :emp_name, :job_id, :mgr_id, :hire_date, :salary, :comm_pct, :dept_id)"
    param_dict = {
        'emp_id': emp_id,
        'emp_name': emp_name,
        'job_id': job_id,
        'mgr_id': mgr_id,
        'hire_date': hire_date,
        'salary': salary,
        'comm_pct': comm_pct,
        'dept_id': dept_id
    }
    
    with cx_Oracle.connect('c##scott_join/tiger@localhost:51521/XE') as conn:
        with conn.cursor() as cursor:
            cursor.execute(insert_sql, param_dict)
            conn.commit()

In [17]:
def insert_emp():
    # 사용자로부터 insert할 값들을 입력 받기 + 검증
    emp_id = int(input('직원 ID: '))
    emp_name = input('이름: ')
    job_id = input('업무 ID: ')
    
    if len(job_id.strip()) == 0:
        job_id = None
        
    try:
        mgr_id = int(input('상사 ID: '))
    except:
        mgr_id = None
        
    hire_date = input('입사일(형식: yyyymmdd): ')
    salary = int(input('급여: '))
    
    try:
        comm_pct = float(input('커미션 비율: '))
        if comm_pct >= 1:
            raise Exception()
    except:
        comm_pct = None
    
    try:
        dept_id = int(input('부서 ID: '))
    except:
        dept_id = None
        
    param = {
        'emp_id': emp_id,
        'emp_name': emp_name,
        'job_id': job_id,
        'mgr_id': mgr_id,
        'hire_date': hire_date,
        'salary': salary,
        'comm_pct': comm_pct,
        'dept_id': dept_id
    }
    
    insert_sql = "INSERT INTO emp VALUES (:emp_id, :emp_name, :job_id, :mgr_id, to_date(:hire_date, 'yyyymmdd'), :salary, :comm_pct, :dept_id)"

    # DB 연결
    with cx_Oracle.connect('c##scott_join/tiger@localhost:51521/XE') as conn:
        with conn.cursor() as cursor:
            cursor.execute(insert_sql, param)
            conn.commit()
            print('INSERT 완료')

In [20]:
insert_emp()

직원 ID: 3500
이름: 이민수
업무 ID: FI_ACCOUNT
상사 ID: 
입사일(형식: yyyymmdd): 20200101
급여: 6000
커미션 비율: 
부서 ID: 
INSERT 완료
